# Check Guild

In [1]:
import os, sys, random
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import engine.models as GM

In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Guild Properties

In [8]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [9]:
guild = GM.Guild.objects.GetOrNone(UserFK=userMd, Name='123')
PD.DataFrame([guild.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,KeepLevel,TotalPower,VaultGold,VaultGems,Selected,LastPlayed,CreateDate
0,1,123,1,0,0,0,True,2023-08-11,2023-08-11


In [13]:
thieves = GM.ThiefInGuild.objects.filter(GuildFK=guild).values()
PD.DataFrame(thieves).drop(['id',
        'Name', 'BasePower', 'Experience', 
        'Position', 'Wounds', 'Cooldown',
        'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', 
#         'Attack', 'Damage', 'Defense',
#         'Traverse', 'Sabotage', 'Perceive', 
        ], axis=1, errors='ignore')

,GuildFK_id,Class,Stars,Level,Power,Agility,Cunning,Might,Endurance,Health,Attack,Damage,Defense,Sabotage,Perceive,Traverse
0,15,Burglar,1,1,70,5,0,0,0,70,5,11,13,5,0,0
1,15,Scoundrel,1,1,70,0,5,0,0,70,2,15,12,0,0,5
2,15,Ruffian,1,1,70,0,0,5,0,70,3,10,16,0,5,0


In [11]:
guildItems = GM.ItemInGuild.objects.filter(GuildFK=guild).values()
PD.DataFrame(guildItems).drop(['id'], axis=1, errors='ignore')

,GuildFK_id,ThiefFK_id,Name,Level,Slot,Power,Requirement,Trait,Attack,Damage,Defense,Skill,Magic
0,15,46,Blackthorn,1,weapon,15,Burglar,agi 1,NaN,11.0,NaN,None,None
1,15,46,Brigandine,1,armor,15,Burglar,agi 1,NaN,NaN,3.0,None,None
2,15,47,Twin Daggers,1,weapon,15,Scoundrel,cun 1,2.0,10.0,NaN,None,None
3,15,47,Vestment,1,armor,15,Scoundrel,cun 1,NaN,NaN,2.0,None,None
4,15,48,Sword & Shield,1,weapon,15,Ruffian,mig 1,3.0,10.0,NaN,None,None
5,15,48,Chain Maille,1,armor,15,Ruffian,mig 1,NaN,NaN,1.0,None,None
6,15,46,Helmet,1,head,10,None,None,NaN,NaN,NaN,per 5,None
7,15,47,Gloves,1,hands,10,None,None,NaN,NaN,NaN,sab 5,None
8,15,48,Boots,1,feet,10,None,None,NaN,NaN,NaN,tra 5,None


### Stages

In [12]:
towers = GM.GuildTower.objects.all().values()
PD.DataFrame(towers).drop(['id'], axis=1, errors='ignore')

""


### Stores